### Sign table

In [10]:
from pyDOE2 import *
# Generate sign table for experiments using Plackett-Burman design
sign_table = pbdesign(3)
sign_table

array([[-1., -1.,  1.],
       [ 1., -1., -1.],
       [-1.,  1., -1.],
       [ 1.,  1.,  1.]])

### Data processing

In [3]:
import pandas as pd
import time
from calendar import timegm
from datetime import datetime

date_format = "%Y/%m/%d:%H:%M:%S"

# Takes a string containing date and time and converts it to epoch time
def timestamp_converter(timestamp):
    utc_time = time.strptime(timestamp, date_format)
    return timegm(utc_time)
    

In [4]:
def load_service_and_arrival_times(experiment):
    service_path ='data/service/exp' + str(experiment) + "/run" 
    arrivals_path ='data/arrivals/exp' + str(experiment) + "/run"

    times = pd.DataFrame({}, columns = ['job', 'arrival_time', 'serivice_time'])
    for run in range(1,4):
        # Load the arrivals and service time data for a single run for this experiment
        temp_service = pd.read_csv(service_path + str(run) + '.csv',  delimiter=' ')
        temp_arrivals = pd.read_csv(arrivals_path + str(run) + '.csv',  delimiter=' ')
        
        # Transform the date string values into epoch values
        temp_arrivals['arrival_time'] = temp_arrivals['arrival_time'].apply(lambda x: timestamp_converter(x))
        
        # Merge the data into a single df
        merged_temp = pd.merge(temp_arrivals, temp_service, on='job')
        
        # Concatenate the df to the results of previous runs 
        if times.empty:
            times = merged_temp
        else:
            times = pd.concat([times, merged_temp])
        
    return times    
    

In [5]:
# Collecting the timing data for all experiments
all_timing_data = {}
for exp in range(1,5):
    # Load the data, remove the job column and reset the row index
    exp_times = load_service_and_arrival_times(exp).drop(columns = ['job']).reset_index(drop=True)
    
    # Compute and add the runtimes
    exp_times['runtime']=exp_times['end_time']-exp_times['arrival_time']
    
    # Add the data for this experiment to all_timing_data
    all_timing_data[exp] = exp_times
    
# all_timing_data.get(1)

In [6]:
# Compute the average runtimes
avg_runtimes = {}
for exp in range(1, 5):
    average_runtime = np.average(np.array(all_timing_data[exp].runtime))
    avg_runtimes[exp] = average_runtime
    print("Average runtime for experiment " + str(exp) + ": " + str(average_runtime))

Average runtime for experiment 1: 124.22859359205815
Average runtime for experiment 2: 1152.8623155487908
Average runtime for experiment 3: 346.8461407661438
Average runtime for experiment 4: 410.5442175292969


#### Accuracies

In [7]:
for exp in range(1,5):
    accuracy_path ='data/accuracies/exp' + str(exp) +".csv" 
    accuracies = pd.read_csv(accuracy_path, header=None)
    print("Average accuracy for experiment " +  str(exp) + ": "+ str(np.average(accuracies)))

Average accuracy for experiment 1: 0.891861403508772
Average accuracy for experiment 2: 0.9598454545454547
Average accuracy for experiment 3: 0.9434787878787879
Average accuracy for experiment 4: 0.944562962962963


### Regresion model

In [45]:
# Create/solve regresion modal
print(sign_table)

y_mean = np.array(list(avg_runtimes.values()))
regression_totals = [sum(avg_runtimes.values())]
for i in range(sign_table.shape[1]):
    regression_totals.append(sum(sign_table[:,i]*y_mean))

print(regression_totals)

# Calulate effect per factor
effect = [x / 4 for x in regression_totals]
print("\nEffect per factor: " + str(effect))

[[-1. -1.  1.]
 [ 1. -1. -1.]
 [-1.  1. -1.]
 [ 1.  1.  1.]]
[2034.4812674362897, 1092.3317987198857, -519.7005508454083, -964.9356451935796]

Effect per factor: [508.62031685907243, 273.0829496799714, -129.92513771135208, -241.2339112983949]


### G/G/1

In [9]:
# Simulate G/G/1 queue
def queuing_simulation(data):
    N = len(data)
    At = data.arrival_time

    S = np.zeros(N) # -> Service start time
    C = np.zeros(N) # -> Complete time
    W = np.zeros(N) # -> response time of job i (waiting time) 

    S[0] = At[0]
    C[0] = S[0] + data.service_time[0]
    W[0] = C[0] - At[0]

    for i in range(1, N):
        S[i] = max(C[i-1], At[i])
        C[i] = S[i] + data.service_time[i]
        W[i] = C[i] - At[i]

#     print("Average simulated run time: " + str(np.average(W)))
    return np.average(W)

In [12]:
# Run the G/G/1 queuing simulation for all experiments
for i in range(1,5):
    # avrg simulated runtime
    print(queuing_simulation(all_timing_data.get(i)))

123.94045613941394
1152.5422222879197
346.7322332938512
410.3491999912262


In [ ]:
#TODO: Compare simulated averages to actual averages